# The Structure of fMRI Data

Before we get started discussing the pre-processing of fMRI data, it is useful to have a review of the structure of an fMRI image. In this lesson we are going to talk a lot about the spatial and temporal elements of fMRI datasets, with lots of references to the fMRI timeseries. So it is important you understand what is meant by this before we go any further.

## 4D images

At its most basic, an fMRI dataset is a collecting of 3D volumes of the brain that were collected in rapid succession. The speed at which we collect these volumes is given by the TR of sequence, and is usually around 2-3 seconds. This gives the temporal resolution of the data. Together, these 3D images form a 4D dataset, where we have three spatial dimensions and a fourth dimension of time. As we move through these 3D volumes we see how the BOLD signal changed over the course of the experiment. Importantly, each of these 3D volumes is collected while the subject is engaged with some sort of cognitive task in the scanner. What these tasks look like and how we use them is the focus of the Experimental Design and Optimisation unit. For the purposes of this lesson, just know that the subject was doing something while in the scanner. Our broad purpose in analysing the data is to try and match changes in the BOLD signal over time with changes in the different conditions of this experimental task.
The fMRI timeseries

Each of the 3D volumes that make up an fMRI dataset can be thought of as a snapshot in time, indicating the values of the BOLD signal at that precise moment. A typical fMRI scan may amass around 200 of these snapshots. For each voxel in the image, we can see how the signal at that point in the image changed over time by looking at how its value shifts across all the 3D volumes. For a scan with 200 volumes, we therefore have 200 values for each voxel in the image documenting how the BOLD signal changed over the course of the experiment. This sequence of values is known as the fMRI timeseries. The timeseries at each voxel represents the raw data that we will subject to statistical analysis to try and discern patterns that map on to the pattern of the experimental conditions. For now, just consider the scale of this data. A typical fMRI volume may have dimensions 60 $\times$ 60 $\times$ 40 with 200 datapoints at each voxel. This gives over 28 million data points in a single fMRI dataset. An example of the timeseries for 3 voxels is shown in {numref}`timeseries-fig`.

```{figure} images/timeseries.png
---
width: 800px
name: timeseries-fig
---
Illustration of how each voxel in a functional image is associated with a timeseries of BOLD signal change.
```

## Example dataset

You can get more of a feel for the structure of fMRI data by downloading the [functional](https://livemanchesterac.sharepoint.com/:u:/r/sites/UOM-FBMH-MSc-NCCN/Shared%20Documents/PCHN62121%20Image%20Analysis/Learning%20Materials/Asynchronous/Week%208/1.%20fMRI%20Pre-processing/Data/IA_preproc_func.nii?csf=1&web=0&e=K2eYv2&download=1) and [anatomical](https://livemanchesterac.sharepoint.com/:u:/r/sites/UOM-FBMH-MSc-NCCN/Shared%20Documents/PCHN62121%20Image%20Analysis/Learning%20Materials/Asynchronous/Week%208/1.%20fMRI%20Pre-processing/Data/IA_preproc_anat.nii?csf=1&web=0&e=cM529F&download=1) files we will be using as examples in this lesson. The video below will show you how to explore this dataset using the SPM facilities for displaying a series of images as a movie, as well as viewing the timeseries at each voxel.

<div style="max-width: 1280px"><div style="position: relative; padding-bottom: 56.25%; height: 0; overflow: hidden;"><iframe src="https://livemanchesterac.sharepoint.com/sites/UOM-FBMH-MSc-NCCN/_layouts/15/embed.aspx?UniqueId=1143354d-b1cf-494f-8c6b-e49588af877a&embed=%7B%22hvm%22%3Atrue%2C%22ust%22%3Atrue%7D&referrer=StreamWebApp&referrerScenario=EmbedDialog.Create" width="1280" height="720" frameborder="0" scrolling="no" allowfullscreen title="1.Exploring-fMRI-Data.mov" style="border:none; position: absolute; top: 0; left: 0; right: 0; bottom: 0; height: 100%; max-width: 100%;"></iframe></div></div>

## The need for pre-processing

This lesson is all about the image processing techniques we use to get our fMRI data ready for statistical analysis. These steps are collectively known as pre-processing, and are needed to get round the various issues that make raw fMRI unsuitable for analysis. Before getting into the details of these issues, examination of the raw fMRI timeseries can actually gives us some intuition about why pre-processing is needed. First of all, consider the GIF in {numref}`functional-flicker-fig`, that shows a set of fMRI volumes being scrolled through as an animation. Notice the characteristic flickering that occurs, which is more noticeable in some voxels compared to others. This flickering represents changes in signal intensity over time. In other words, it represents signal variance, the degree to which the signal moves around over time.

```{figure} images/functional-flicker.gif
---
width: 800px
name: functional-flicker-fig
---
Illustration of the variance at each voxel over time.
```

This variance is the most basic information the signal contains, and is the element that we want to try and associate with our experiment. In other words, we want our experiment to cause changes in the value of the signal. Unfortunately, there are many other reasons why the signal may be changing over time, other than true hemodynamic changes associated with our experiment. These reasons could relate to noise from the scanner, motion from the subject or other aspects of physiology not connected to blood flow. Whatever the reasons, we really want to minimise the extent to which the signal variance is attributable to sources other than our experimental manipulation. This is why, in part, we need pre-processing to help clean the data, to make it possible to see the changes we want to see and minimise everything else.

Beyond minimising sources of noise and additional variance, pre-processing also aims to address a fundamental difficulty with multi-subject fMRI studies, namely that everyone's brain is a different shape and size. This anatomical variability can be substantial, as shown in {numref}`anat-var-fig`, and causes issues with the analysis and localisation of groups of scans. So unless you are only performing an analysis on a single individual, this needs to be addressed. We already discussed the principles of normalisation in Functional Neuroanatomy, and the idea here are the same, just applied to functional data instead of structural. So the other aim of pre-processing is therefore to get all functional data from all subjects into the same space, to allow for localisation of results and the analysis of groups of scans.

```{figure} images/anat-var.png
---
width: 800px
name: anat-var-fig
---
Illustration of the variability in anatomy across different individuals.
```

## Pre-processing preliminaries

Before diving into the standard pre-processing steps, there is some housekeeping to do. Firstly, it is always better to work on copies of your data rather than the original files. Not only does this make it easier to delete everything and start over if there are problems, but SPM can also make some invisible changes to the image headers that you may want to remove before starting again. Working with new copies each time is the easiest way to do this. SPM also makes a lot of new files during pre-processing and so it is useful to create a new folder to keep things organised. As such, the first step is often to make a new folder and then make copies of the images for processing inside that folder.

The next thing we need to do is to look at the default registration between our functional and structural images. Many of the pre-processing steps we use are based on image registration and, as we discussed during Functional Neuroanatomy, we can help the registration algorithms by doing some rough manual alignment to start with. So, for our example data, we can first set the origins of the functional and structural scans to the anterior commissure, and then make any adjustments we need to bring them into better registration before we begin.

Both of these preliminary steps are demonstrated in the video below.

<div style="max-width: 1280px"><div style="position: relative; padding-bottom: 56.25%; height: 0; overflow: hidden;"><iframe src="https://livemanchesterac.sharepoint.com/sites/UOM-FBMH-MSc-NCCN/_layouts/15/embed.aspx?UniqueId=ddf398ee-04b8-4401-9215-26a989ae247c&embed=%7B%22hvm%22%3Atrue%2C%22ust%22%3Atrue%7D&referrer=StreamWebApp&referrerScenario=EmbedDialog.Create" width="1280" height="720" frameborder="0" scrolling="no" allowfullscreen title="2.Preprocessing-Preliminaries.mov" style="border:none; position: absolute; top: 0; left: 0; right: 0; bottom: 0; height: 100%; max-width: 100%;"></iframe></div></div>